# 1. Table extraction

In [1]:
import logging
import pathlib

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
from esg_data_pipeline import PDFTableExtractor
from esg_data_pipeline import config

In [3]:
PDFTableExtractor_kwargs = {
    "batch_size": -1,
    "cscdtabnet_config": config.CONFIG_FOLDER / "cascade_mask_rcnn_hrnetv2p_w32_20e.py",
    "cscdtabnet_ckpt": config.CHECKPOINT_FOLDER / "icdar_19b2.pth",
    "bbox_thres": 0.85,
    "dpi": 200,
}

In [4]:
table_extractor = PDFTableExtractor(**PDFTableExtractor_kwargs)

2020-08-17 20:33:03,398 — esg_data_pipeline.components.pdf_table_extractor — INFO —__create_model:70 — cascadetabnet checkpoint does not exist. Downloading...


Downloading...
From: https://drive.google.com/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT
To: /esg_data_pipeline/esg_data_pipeline/checkpoint/icdar_19b2.pth
39.3MB [00:00, 68.3MB/s]('Connection broken: OSError("(104, \'ECONNRESET\')")', OSError("(104, 'ECONNRESET')"))
50.3MB [00:00, 97.4MB/s]


2020-08-17 20:33:06,031 — esg_data_pipeline.components.pdf_table_extractor — INFO —__create_model:85 — Error while downloading cascadetabnet checkpoint. Redownloading...


Downloading...
From: https://drive.google.com/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT
To: /esg_data_pipeline/esg_data_pipeline/checkpoint/icdar_19b2.pth
664MB [00:03, 212MB/s] 


### Extract tables from a single pdf file
Table Extraction happens in two stages:
1. **Infer Bounding Box:** Convert a pdf into image and use an object detection model for table recognition. The model is called CascadeTabNet. The output of this stage is the coordinates of tables.

2. **Extract tables:** from PDFs using Tabula. The coordinates are passed to tabula for analyzing that area of page.

### Automate these processes using run_folder() function

We extract all tables from pdfs in the `data/pdfs` folder. First, let's delete all files in the extraction folder. In the data/pdfs folder, we have a pdf `NYSE_TOT_2015 annual.pdf`

In [5]:
!ls $config.PDF_FOLDER

'NYSE_TOT_2015 annual.pdf'


In [6]:
!rm -rf $config.EXTRACTION_FOLDER/*

In [7]:
!ls $config.EXTRACTION_FOLDER

In [8]:
table_extractor.run_folder(config.PDF_FOLDER, config.EXTRACTION_FOLDER)

2020-08-17 20:33:25,345 — esg_data_pipeline.components.pdf_table_extractor — INFO —run:273 — PDFTableExtractor is running on file /esg_data_pipeline/esg_data_pipeline/data/pdfs/NYSE_TOT_2015 annual.pdf...


Inferring tables for page 1-295:   0%|          | 0/294 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))

Inferring tables for page 1-295:  31%|███       | 90/294 [00:28<01:30,  2.26it/s]


Inferring tables for page 1-295:  63%|██████▎   | 185/294 [00:59<01:01,  1.79it/s]


Inferring tables for page 1-295:  95%|█████████▌| 280/294 [02:10<00:09,  1.52it/s]


Page: 100%|█████████▉| 294/295 [05:40<00:01,  1.16s/it]
Inferring tables for page 295-589: 0it [00:00, ?it/s]
Page: 588it [05:41,  1.23it/s]                         
Extracting and saving tables: 100%|██████████| 167/167 [00:00<00:00, 196039.40it/s]
Page: 588it [06:00,  1.23it/s]

Each detected tables is saved as a CSV file.

In [9]:
!ls $config.EXTRACTION_FOLDER

'NYSE_TOT_2015 annual_page102_1.csv'  'NYSE_TOT_2015 annual_page225_2.csv'
'NYSE_TOT_2015 annual_page104_1.csv'  'NYSE_TOT_2015 annual_page225_3.csv'
'NYSE_TOT_2015 annual_page105_1.csv'  'NYSE_TOT_2015 annual_page226_1.csv'
'NYSE_TOT_2015 annual_page108_1.csv'  'NYSE_TOT_2015 annual_page226_2.csv'
'NYSE_TOT_2015 annual_page109_1.csv'  'NYSE_TOT_2015 annual_page226_3.csv'
'NYSE_TOT_2015 annual_page109_2.csv'  'NYSE_TOT_2015 annual_page226_4.csv'
'NYSE_TOT_2015 annual_page110_1.csv'  'NYSE_TOT_2015 annual_page227_1.csv'
'NYSE_TOT_2015 annual_page110_2.csv'  'NYSE_TOT_2015 annual_page228_1.csv'
'NYSE_TOT_2015 annual_page112_1.csv'  'NYSE_TOT_2015 annual_page229_1.csv'
'NYSE_TOT_2015 annual_page112_2.csv'  'NYSE_TOT_2015 annual_page22_1.csv'
'NYSE_TOT_2015 annual_page113_1.csv'  'NYSE_TOT_2015 annual_page230_1.csv'
'NYSE_TOT_2015 annual_page113_2.csv'  'NYSE_TOT_2015 annual_page230_2.csv'
'NYSE_TOT_2015 annual_page114_1.csv'  'NYSE_TOT_2015 annual_page230_3.csv'
'NYSE_TOT_2015 annual_page

# 2. Table Curation

We will be using Allyson's annotations on this report `NYSE_TOT_2015 annual.pdf`. The annotation file exists in `config.ANNOTATION_FOLDER`.

In [10]:
from esg_data_pipeline import TableCurator

In [11]:
tb_cur = TableCurator(
    neg_pos_ratio=1,
    create_neg_samples=True,
    columns_to_read=["company", "source_file", "source_page", "kpi_id", "year", "answer", "data_type"],
    company_to_exclude=["CEZ"],
)

In [12]:
import glob

annotation_excels = glob.glob("{}/[!~$]*[.xlsx]".format(config.ANNOTATION_FOLDER))
tb_cur.run(config.EXTRACTION_FOLDER, annotation_excels, config.CURATION_FOLDER)

2020-08-17 20:41:38,547 — esg_data_pipeline.components.table_curator — WARNING —__clean_annotation_file:293 — File 1qbit_edited_kpi_extraction template_Allyson.xlsx has invalid kpis: [13.0, 2.1, 2.2]
2020-08-17 20:41:38,559 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — 2015_BASF_Report.pdf was not extracted.
2020-08-17 20:41:38,560 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — 2015_BASF_Report.pdf was not extracted.
2020-08-17 20:41:38,561 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — 2015_BASF_Report.pdf was not extracted.
2020-08-17 20:41:38,562 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — 2015_BASF_Report.pdf was not extracted.
2020-08-17 20:41:38,563 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — 2015_BASF_Report.pdf was not extracted.
2020-08-17 20:4

2020-08-17 20:41:38,601 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual_report_2019_eng.pdf was not extracted.
2020-08-17 20:41:38,602 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual_report_2019_eng.pdf was not extracted.
2020-08-17 20:41:38,602 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual_report_2019_eng.pdf was not extracted.
2020-08-17 20:41:38,603 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual 2018.pdf was not extracted.
2020-08-17 20:41:38,604 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual 2018.pdf was not extracted.
2020-08-17 20:41:38,605 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual 2018.pdf was not extracted.
2020-08-17 20:41:38,606 — esg_data_pipeline.components.

2020-08-17 20:41:38,641 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:41:38,642 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:41:38,643 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:41:38,644 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:41:38,645 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:41:38,645 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Aker-BP-Annual-report-2018.pdf was not extracted.
2020-08-17 20:41:38,646 — esg_dat

2020-08-17 20:41:38,683 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:41:38,684 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:41:38,685 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:41:38,686 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:41:38,687 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:41:38,688 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020

2020-08-17 20:41:38,723 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020-08-17 20:41:38,724 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020-08-17 20:41:38,724 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren Corporation Climate Risk Report 2019.pdf was not extracted.
2020-08-17 20:41:38,725 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren Corporation Climate Risk Report 2019.pdf was not extracted.
2020-08-17 20:41:38,726 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren Corporation Climate Risk Report 2019.pdf was not extracted.
2020-08-17 20:41:38,727 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 —

...

2020-08-17 20:41:38,781 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Adani Group Adani Enterprises Annual Report 2019.pdf was not extracted.
2020-08-17 20:41:38,782 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Adani Group Adani Enterprises Annual Report 2019.pdf was not extracted.
2020-08-17 20:41:38,783 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Adani Group Adani Enterprises Annual Report 2019.pdf was not extracted.
2020-08-17 20:41:38,784 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Adani Groupl Adani Enterprises Sustainability Report 2019.pdf was not extracted.
2020-08-17 20:41:38,785 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Adani Groupl Adani Enterprises Sustainability Report 2019.pdf was not extracted.
2020-08-17 20:41:38,786 — esg_data_pi

2020-08-17 20:41:38,848 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:41:38,849 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:41:38,850 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:41:38,850 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:41:38,851 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:41:38,852 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:41:38,853 — esg_data

2020-08-17 20:41:38,889 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:41:38,890 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:41:38,890 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:41:38,891 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:41:38,892 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:41:38,893 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:41:38,893 — esg_data

2020-08-17 20:41:38,930 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:41:38,931 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:41:38,932 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:41:38,933 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:41:38,933 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:41:38,934 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:41:38,935 — esg_data

2020-08-17 20:41:38,971 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual 2017.pdf was not extracted.
2020-08-17 20:41:38,972 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual 2017.pdf was not extracted.
2020-08-17 20:41:38,973 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual 2017.pdf was not extracted.
2020-08-17 20:41:38,974 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual 2017.pdf was not extracted.


Most of the warnings are just pdfs mentioned in the excel file which were not extracted, in other words do not exist in the initial pdfs folder. Next, we will see that the curation folder will have a csv which shows the curated data.

In [13]:
!ls $config.CURATION_FOLDER

esg_TABLE_dataset.csv


In [14]:
import pandas as pd

df = pd.read_csv(config.CURATION_FOLDER / "esg_TABLE_dataset.csv", index_col=0)
df.head()

,Company,Year,Question,Answer,Table_filename,Label
0,Total SA,2013,What is the volume of estimated proven or prob...,NaN,NYSE_TOT_2015 annual_page242_4.csv,0
1,Total SA,2013,What is the volume of estimated proven or prob...,"11,526 million barrels of oil equivalent",NYSE_TOT_2015 annual_page266_1.csv,1
2,Total SA,2014,What is the volume of estimated proven or prob...,NaN,NYSE_TOT_2015 annual_page230_2.csv,0
3,Total SA,2014,What is the volume of estimated proven or prob...,"11,523 million barrels of oil equivalent",NYSE_TOT_2015 annual_page266_1.csv,1
4,Total SA,2015,What is the volume of estimated proven or prob...,NaN,NYSE_TOT_2015 annual_page8_1.csv,0


# 3. Text Extraction

The text extraction stage extracts all the paragraphs in a pdf and saves them in a JSON file, so that it can be easily handled by Python.

This process is applied to all the pdfs mentioned in an annotation Excel file provided by Allianz.
A JSON file is created for each pdf, with the same file name.
Place the Excel files in data/annotations, and the pdfs in data/pdfs.
The extracted pdfs will be saved in data/extraction. These directories can be changed in the config file config/config.py

In [15]:
import os

from esg_data_pipeline import config
from esg_data_pipeline.components import PDFTextExtractor
from esg_data_pipeline.components import TextCurator

In [16]:
config.PDFTextExtractor_kwargs

{'min_paragraph_length': 30,
 'annotation_folder': PosixPath('/esg_data_pipeline/esg_data_pipeline/data/annotations')}

In [17]:
ext = PDFTextExtractor(**config.PDFTextExtractor_kwargs)
ext.run_folder(config.PDF_FOLDER, config.EXTRACTION_FOLDER)

2020-08-17 20:42:22,506 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf 2015_BASF_Report.pdf
2020-08-17 20:42:22,508 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf BASF_Report_2016.pdf
2020-08-17 20:42:22,510 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf BASF_Report_2017.pdf
2020-08-17 20:42:22,511 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf BASF_Report_2018.pdf
2020-08-17 20:42:22,513 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf Wintershall Dea annual report 2019.pdf
2020-08-17 20:42:22,514 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf Wintershall-Dea_Sustainability_Report_2019.pdf
2020-08-17 20:42:22,516 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not

/opt/conda/lib/python3.7/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='/esg_data_pipeline/esg_data_pipeline/data/pdfs/NYSE_TOT_2015 annual.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


2020-08-17 20:43:29,444 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf NYSE_TOT_2016 annual.pdf
2020-08-17 20:43:29,446 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf NYSE_TOT_2017 annual.pdf
2020-08-17 20:43:29,447 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf NYSE_TOT_2018 annual.pdf
2020-08-17 20:43:29,449 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf Transocean_Sustain_digital_FN_4 2017_2018.pdf
2020-08-17 20:43:29,450 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf sustainable development 2017.pdf
2020-08-17 20:43:29,451 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf annual 2018 .pdf
2020-08-17 20:43:29,452 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could no

We can see that the JSON files are created for each pdf.

In [18]:
!ls -lh $config.EXTRACTION_FOLDER

total 2.4M
-rw-r--r-- 1 root root 1.3M Aug 17 20:43 'NYSE_TOT_2015 annual.json'
-rw-r--r-- 1 root root 2.1K Aug 17 20:39 'NYSE_TOT_2015 annual_page102_1.csv'
-rw-r--r-- 1 root root  311 Aug 17 20:39 'NYSE_TOT_2015 annual_page104_1.csv'
-rw-r--r-- 1 root root  734 Aug 17 20:39 'NYSE_TOT_2015 annual_page105_1.csv'
-rw-r--r-- 1 root root 1.8K Aug 17 20:39 'NYSE_TOT_2015 annual_page108_1.csv'
-rw-r--r-- 1 root root  539 Aug 17 20:39 'NYSE_TOT_2015 annual_page109_1.csv'
-rw-r--r-- 1 root root 1.5K Aug 17 20:39 'NYSE_TOT_2015 annual_page109_2.csv'
-rw-r--r-- 1 root root  783 Aug 17 20:39 'NYSE_TOT_2015 annual_page110_1.csv'
-rw-r--r-- 1 root root 1.4K Aug 17 20:39 'NYSE_TOT_2015 annual_page110_2.csv'
-rw-r--r-- 1 root root 1.8K Aug 17 20:39 'NYSE_TOT_2015 annual_page112_1.csv'
-rw-r--r-- 1 root root  296 Aug 17 20:39 'NYSE_TOT_2015 annual_page112_2.csv'
-rw-r--r-- 1 root root  465 Aug 17 20:39 'NYSE_TOT_2015 annual_page113_1.csv'
-rw-r--r-- 1 root root 1.8K Aug 17 20:39 'NYSE_TOT_2015 annual

**Alternatively**, we can have the pipeline work on a **single pdf**, by using the `run()` method and specifying the path to the desired pdf file.
Let's create a test directory and save the JSON file in there.

In [19]:
test_dir = "{}/test_dir".format(config.DATA_FOLDER)
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

sample_pdf = "{}/NYSE_TOT_2015 annual.pdf".format(config.PDF_FOLDER)
text_dict = ext.run(input_filepath=sample_pdf, output_folder=test_dir)

In [20]:
!ls $config.DATA_FOLDER/test_dir

'NYSE_TOT_2015 annual.json'


# 4. Text Curation


The extracted JSON files are fed into the next stage to curate a training dataset.
<br>The positive examples (label 1) are taken from the annotated data provided by Allinaz.
<br>A negative example (label 0) for each question is created by selecting a random paragraph from the JSON files.

In [21]:
ls $config.CURATION_FOLDER/*

/esg_data_pipeline/esg_data_pipeline/data/curation/esg_TABLE_dataset.csv


In [22]:
cur = TextCurator(**config.TextCurator_kwargs)
cur.run(config.EXTRACTION_FOLDER, annotation_excels, config.CURATION_FOLDER)

2020-08-17 20:45:28,367 — esg_data_pipeline.components.text_curator — WARNING —process_relevant_sentences:256 — Could not process row number 101 in 1qbit_edited_kpi_extraction template_Allyson.xlsx
2020-08-17 20:45:29,140 — esg_data_pipeline.components.text_curator — WARNING —run:79 — The corresponding KPIs can not be mapped             to any questions and the mapped question is empty
[13.]
2020-08-17 20:45:29,151 — esg_data_pipeline.components.text_curator — INFO —run:91 — Curated 215 examples
2020-08-17 20:45:29,151 — esg_data_pipeline.components.text_curator — INFO —run:92 — Saving the dataset in /esg_data_pipeline/esg_data_pipeline/data/curation/esg_TEXT_dataset.csv


In [23]:
ls $config.CURATION_FOLDER/*

/esg_data_pipeline/esg_data_pipeline/data/curation/esg_TABLE_dataset.csv
/esg_data_pipeline/esg_data_pipeline/data/curation/esg_TEXT_dataset.csv


# 5. Extraction pipeline

We can create an extraction pipeline using `Extractor` object to run extraction of text and table data. In the config file, we define the extractors which we would like to run on the pdfs.

In [24]:
config.EXTRACTORS

[('PDFTableExtractor',
  {'batch_size': -1,
   'cscdtabnet_config': PosixPath('/esg_data_pipeline/esg_data_pipeline/config/cascade_mask_rcnn_hrnetv2p_w32_20e.py'),
   'cscdtabnet_ckpt': PosixPath('/esg_data_pipeline/esg_data_pipeline/checkpoint/icdar_19b2.pth'),
   'bbox_thres': 0.85,
   'dpi': 200}),
 ('PDFTextExtractor',
  {'min_paragraph_length': 30,
   'annotation_folder': PosixPath('/esg_data_pipeline/esg_data_pipeline/data/annotations')})]

In [25]:
from esg_data_pipeline.components import Extractor

ext = Extractor(config.EXTRACTORS)

In [26]:
ext.run_folder(config.PDF_FOLDER, config.EXTRACTION_FOLDER)

2020-08-17 20:45:52,122 — esg_data_pipeline.components.pdf_table_extractor — INFO —run:273 — PDFTableExtractor is running on file /esg_data_pipeline/esg_data_pipeline/data/pdfs/NYSE_TOT_2015 annual.pdf...



Page:   0%|          | 0/295 [00:00<?, ?it/s]

Inferring tables for page 1-295:   0%|          | 0/294 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Inferring tables for page 1-295:   0%|          | 1/294 [00:00<01:58,  2.47it/s]

Inferring tables for page 1-295:   1%|          | 2/294 [00:00<01:35,  3.06it/s]

Inferring tables for page 1-295:   1%|          | 3/294 [00:01<01:53,  2.57it/s]

Inferring tables for page 1-295:   1%|▏         | 4/294 [00:01<01:52,  2.59it/s]

Inferring tables for page 1-295:   2%|▏         | 5/294 [00:01<01:55,  2.51it/s]

Inferring tables for page 1-295:   2%|▏         | 6/294 [00:02<01:32,  3.11it/s]

Inferring tables for page 1

Inferring tables for page 1-295:  30%|██▉       | 87/294 [00:27<01:39,  2.08it/s]

Inferring tables for page 1-295:  30%|██▉       | 88/294 [00:27<01:18,  2.63it/s]

Inferring tables for page 1-295:  30%|███       | 89/294 [00:27<01:25,  2.41it/s]

Inferring tables for page 1-295:  31%|███       | 90/294 [00:28<01:32,  2.20it/s]

Inferring tables for page 1-295:  31%|███       | 91/294 [00:28<01:13,  2.77it/s]

Inferring tables for page 1-295:  31%|███▏      | 92/294 [00:28<00:59,  3.37it/s]

Inferring tables for page 1-295:  32%|███▏      | 93/294 [00:28<00:51,  3.88it/s]

Inferring tables for page 1-295:  32%|███▏      | 94/294 [00:29<00:45,  4.44it/s]

Inferring tables for page 1-295:  32%|███▏      | 95/294 [00:29<00:40,  4.95it/s]

Inferring tables for page 1-295:  33%|███▎      | 96/294 [00:29<00:36,  5.40it/s]

Inferring tables for page 1-295:  33%|███▎      | 97/294 [00:29<00:34,  5.76it/s]

Inferring tables for page 1-295:  33%|███▎      | 98/294 [00:29<00:32,  6.06it/s]

Infe

Inferring tables for page 1-295:  61%|██████    | 178/294 [00:56<00:31,  3.72it/s]

Inferring tables for page 1-295:  61%|██████    | 179/294 [00:57<00:27,  4.26it/s]

Inferring tables for page 1-295:  61%|██████    | 180/294 [00:57<00:23,  4.85it/s]

Inferring tables for page 1-295:  62%|██████▏   | 181/294 [00:57<00:22,  4.94it/s]

Inferring tables for page 1-295:  62%|██████▏   | 182/294 [00:57<00:22,  5.07it/s]

Inferring tables for page 1-295:  62%|██████▏   | 183/294 [00:57<00:24,  4.56it/s]

Inferring tables for page 1-295:  63%|██████▎   | 184/294 [00:58<00:45,  2.43it/s]

Inferring tables for page 1-295:  63%|██████▎   | 185/294 [00:59<01:02,  1.75it/s]

Inferring tables for page 1-295:  63%|██████▎   | 186/294 [01:00<01:12,  1.49it/s]

Inferring tables for page 1-295:  64%|██████▎   | 187/294 [01:01<01:20,  1.33it/s]

Inferring tables for page 1-295:  64%|██████▍   | 188/294 [01:02<01:23,  1.28it/s]

Inferring tables for page 1-295:  64%|██████▍   | 189/294 [01:03<01:28,  1.1

Inferring tables for page 1-295:  91%|█████████▏| 269/294 [02:02<00:19,  1.26it/s]

Inferring tables for page 1-295:  92%|█████████▏| 270/294 [02:03<00:20,  1.20it/s]

Inferring tables for page 1-295:  92%|█████████▏| 271/294 [02:04<00:19,  1.15it/s]

Inferring tables for page 1-295:  93%|█████████▎| 272/294 [02:05<00:19,  1.12it/s]

Inferring tables for page 1-295:  93%|█████████▎| 273/294 [02:06<00:19,  1.10it/s]

Inferring tables for page 1-295:  93%|█████████▎| 274/294 [02:07<00:18,  1.09it/s]

Inferring tables for page 1-295:  94%|█████████▎| 275/294 [02:08<00:17,  1.08it/s]

Inferring tables for page 1-295:  94%|█████████▍| 276/294 [02:09<00:16,  1.07it/s]

Inferring tables for page 1-295:  94%|█████████▍| 277/294 [02:10<00:16,  1.06it/s]

Inferring tables for page 1-295:  95%|█████████▍| 278/294 [02:11<00:15,  1.05it/s]

Inferring tables for page 1-295:  95%|█████████▍| 279/294 [02:12<00:13,  1.12it/s]

Inferring tables for page 1-295:  95%|█████████▌| 280/294 [02:12<00:09,  1.4

2020-08-17 20:53:54,095 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf 2015_BASF_Report.pdf
2020-08-17 20:53:54,096 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf BASF_Report_2016.pdf
2020-08-17 20:53:54,097 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf BASF_Report_2017.pdf
2020-08-17 20:53:54,098 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf BASF_Report_2018.pdf
2020-08-17 20:53:54,100 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf Wintershall Dea annual report 2019.pdf
2020-08-17 20:53:54,102 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf Wintershall-Dea_Sustainability_Report_2019.pdf
2020-08-17 20:53:54,103 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not

/opt/conda/lib/python3.7/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='/esg_data_pipeline/esg_data_pipeline/data/pdfs/NYSE_TOT_2015 annual.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


2020-08-17 20:55:02,085 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf NYSE_TOT_2016 annual.pdf
2020-08-17 20:55:02,087 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf NYSE_TOT_2017 annual.pdf
2020-08-17 20:55:02,088 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf NYSE_TOT_2018 annual.pdf
2020-08-17 20:55:02,090 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf Transocean_Sustain_digital_FN_4 2017_2018.pdf
2020-08-17 20:55:02,091 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf sustainable development 2017.pdf
2020-08-17 20:55:02,093 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could not find pdf annual 2018 .pdf
2020-08-17 20:55:02,094 — esg_data_pipeline.components.pdf_text_extractor — INFO —get_path_pdf:104 — Could no

# 6. Curation pipeline

Similarly, we can create a curation pipeline using `Curator` object to run curation of both text and table data. In the config file, we define the curators.

In [27]:
config.CURATORS

[('TextCurator',
  {'retrieve_paragraph': False,
   'neg_pos_ratio': 1,
   'create_neg_samples': False}),
 ('TableCurator',
  {'neg_pos_ratio': 1,
   'create_neg_samples': True,
   'columns_to_read': ['company',
    'source_file',
    'source_page',
    'kpi_id',
    'year',
    'answer',
    'data_type'],
   'company_to_exclude': ['CEZ']})]

In [28]:
from esg_data_pipeline.components import Curator

cur = Curator(config.CURATORS)

In [29]:
cur.run(config.EXTRACTION_FOLDER, config.ANNOTATION_FOLDER, config.CURATION_FOLDER)

2020-08-17 20:56:07,500 — esg_data_pipeline.components.curator — INFO —run:45 — Received 1 excel files
2020-08-17 20:56:07,769 — esg_data_pipeline.components.text_curator — WARNING —process_relevant_sentences:256 — Could not process row number 101 in 1qbit_edited_kpi_extraction template_Allyson.xlsx
2020-08-17 20:56:08,561 — esg_data_pipeline.components.text_curator — WARNING —run:79 — The corresponding KPIs can not be mapped             to any questions and the mapped question is empty
[13.]
2020-08-17 20:56:08,572 — esg_data_pipeline.components.text_curator — INFO —run:91 — Curated 215 examples
2020-08-17 20:56:08,573 — esg_data_pipeline.components.text_curator — INFO —run:92 — Saving the dataset in /esg_data_pipeline/esg_data_pipeline/data/curation/esg_TEXT_dataset.csv
2020-08-17 20:56:09,482 — esg_data_pipeline.components.table_curator — WARNING —__clean_annotation_file:293 — File 1qbit_edited_kpi_extraction template_Allyson.xlsx has invalid kpis: [13.0, 2.1, 2.2]
2020-08-17 20:56:

2020-08-17 20:56:09,540 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Wintershall-Dea_Sustainability_Report_2019.pdf was not extracted.
2020-08-17 20:56:09,541 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Wintershall-Dea_Sustainability_Report_2019.pdf was not extracted.
2020-08-17 20:56:09,544 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual_report_2019_eng.pdf was not extracted.
2020-08-17 20:56:09,545 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual_report_2019_eng.pdf was not extracted.
2020-08-17 20:56:09,546 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual_report_2019_eng.pdf was not extracted.
2020-08-17 20:56:09,547 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — annual_report_2019_eng.pdf w

2020-08-17 20:56:09,593 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — RN_SR_2016_EN(2) sustainabilitz 2016.pdf was not extracted.
2020-08-17 20:56:09,594 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:56:09,595 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:56:09,596 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:56:09,597 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:56:09,597 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — sustainability 2015.pdf was not extracted.
2020-08-17 20:56:09,598

2020-08-17 20:56:09,634 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:56:09,635 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:56:09,636 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:56:09,636 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:56:09,637 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020-08-17 20:56:09,638 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AKERBP-Annual-Report-2016.pdf was not extracted.
2020

2020-08-17 20:56:09,672 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020-08-17 20:56:09,673 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020-08-17 20:56:09,673 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020-08-17 20:56:09,674 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020-08-17 20:56:09,675 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020-08-17 20:56:09,676 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — Ameren_2019_Annual_Report.pdf was not extracted.
2020

...

2020-08-17 20:56:09,716 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AGL Energy Ltd FY19 Carbon Scenario Analysis.pdf was not extracted.
2020-08-17 20:56:09,717 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AGL Energy Ltd FY19 Carbon Scenario Analysis.pdf was not extracted.
2020-08-17 20:56:09,718 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AGL Energy Ltd FY19 Carbon Scenario Analysis.pdf was not extracted.
2020-08-17 20:56:09,719 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AGL Energy Ltd FY19 Carbon Scenario Analysis.pdf was not extracted.
2020-08-17 20:56:09,719 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — AGL Energy Ltd FY19 Carbon Scenario Analysis.pdf was not extracted.
2020-08-17 20:56:09,720 — esg_data_pipeline.components.table_curator — WARN

2020-08-17 20:56:09,778 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:56:09,779 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:56:09,780 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:56:09,780 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:56:09,781 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:56:09,782 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2016 annual.pdf was not extracted.
2020-08-17 20:56:09,783 — esg_data

2020-08-17 20:56:09,820 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:56:09,821 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:56:09,821 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:56:09,822 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:56:09,823 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:56:09,824 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2017 annual.pdf was not extracted.
2020-08-17 20:56:09,825 — esg_data

2020-08-17 20:56:09,863 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,864 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,865 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,866 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,867 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,867 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,868 — esg_data

2020-08-17 20:56:09,905 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,906 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,907 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,908 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,908 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,909 — esg_data_pipeline.components.table_curator — WARNING —process_single_annotation_file:398 — NYSE_TOT_2018 annual.pdf was not extracted.
2020-08-17 20:56:09,910 — esg_data